### Том хэлний загвар Монгол хэлэн дээр GRPO механизм ашиглан сургах: LLM 8B GRPO train on MGL dataset

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1e1W1mbGDB_Yjhof4dVqZdVi5YC3VLCqP?usp=sharing)

<div class="align-center">
  <a href="https://www.youtube.com/@deeplearning_mn"><img src="https://www.svgrepo.com/show/13671/youtube.svg" height="50"></a>
     <a href="https://www.facebook.com/groups/1026243374478030"><img src="https://www.svgrepo.com/show/475647/facebook-color.svg" height="45"></a>
 <a href="https://docs.google.com/forms/d/e/1FAIpQLSetEXXZP89FrkST-ox4PDRfDCvbResPN3FKoNmJJkq_hKv8Qw/viewform"><img src="https://www.svgrepo.com/show/324465/neuron-connections-neural-network.svg" height="50"></a>


 ⭐️ <i>Star us on <a href="https://github.com/deeplearningmn/AIEng">Github</a> </i> ⭐️
</div>

### Installation

Өгөгдөл татах линк

https://github.com/deeplearningmn/AIEng/blob/main/7.genai/mgl_grpo_dataset.parquet

Дараа нь mgl_grpo_dataset.parquet өгөгдлөө колаб  content/ руу хуулаарай

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### AIEng program GenAI LLM

Load up `Llama 3.1 8B Instruct`, and set parameters

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-17 10:00:17 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-17 10:00:18 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.5: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.43%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0.

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

INFO 05-17 10:03:48 [model_runner.py:1592] Graph capturing finished in 50 secs, took 0.53 GiB
INFO 05-17 10:03:48 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 62.28 seconds
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'k_norm', 'q_norm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'k_norm', 'q_norm', 'post_feedforward_layernorm']


tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.5.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Operation (ETL)


In [ ]:
dataset_file = '/content/mgl_grpo_dataset.parquet'
# data = load_dataset(dataset_name,'data',cache_dir=dataset_name)
# print(data)
import pandas as pd
from datasets import load_dataset, Dataset
import math
data = pd.read_parquet(dataset_file)

hf_dataset = Dataset.from_pandas(data)
hf_dataset = hf_dataset.filter(lambda d:  d["input"]  not in [None, "nan", ""] and d["output"]  not in [None, "nan", ""] and d["instruction"]  not in [None, "nan", ""])

# data = hf_dataset.map(lambda x:
#        print(x['input'])
#  )
number_of_data = 9_000
hf_dataset = hf_dataset.select(range(number_of_data))
print(hf_dataset)

Filter:   0%|          | 0/41601 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 9000
})


In [ ]:
import re
from datasets import load_dataset, Dataset



# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

dataset_question_field = "instruction" #question
dataset_answer_field = "output" #answer

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_mgl_questions(split = "train") -> Dataset:
    # data = load_dataset('saillab/alpaca-mongolian-cleaned', 'main')[split] # type: ignore
    data = hf_dataset.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['instruction']}
        ],
        'answer': extract_hash_answer(x['output']),
        'question': x['instruction']
    }) # type: ignore

    print('data :',data)
    return data # type: ignore

dataset = get_mgl_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

data : Dataset({
    features: ['instruction', 'input', 'output', 'prompt', 'answer', 'question'],
    num_rows: 9000
})


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
max_prompt_length = 256


from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 6


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,000 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 1 x 1) = 6
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)


-------------------- Question:
 Өгүүлбэрийг цэг таслал, том үсгээр зөв бичихийн тулд засна уу 
Answer:
None 
Response:
Тухайн салбарын 3 шинэ чиг хандлагийн жагсаалт нь сансрын нэмэх мэддэг, түүхтэй гэрчилгээг тус тус шилтгэхад, улмаар нөөц арилжааны хөгжимсэн дутагдалаас сэргээн шийдэх чиг хандлаг болон салбарын үйл ажиллагааг нэмэх чиг хандлаг, жинхэрээ эрэлхэг байдалтай чиг хандлагийг д.HTTP хангамгийн шилдэг хэлбэрүүдийн хлоо хардаж тоног төхөөрөл байгууламжуудад тогтмол бэлдсэн дутагдашгүй хангамжыг нэмээж, салбарын байгууламжны сансрэл хөгжүүм сэргээхгүй хандлагийн нэг байгууламжыг тус тус хангамгийн үйл ажиллагаа самбараасаа ажилуулж, тус байгууламжын хэмжээс зайгаа тохирох хадгалж интерфэйсийн зөвхөн хэмжээс зайг тогтруулж, эх нээлтийн хангалт байртан дацигба хамагтай, тэр дотоод байгууламжын шииднүүдийг шилжих үүрэлтгэл байдаг хэрэгтэй ажиллалттай, тоногийн хяналттай салбарын гүйцэттэй байгууламж нэгийг хэвийгт үйл ажиллагаа явуулах зорилгод орлоо.



Тухайн салбарт шинэ чиг х

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,-0.000000,-0.112000,0.183006,480.166687,0.000000,-0.112000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,260.666687,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,411.333344,0.000012,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,-0.090833,0.362666,602.500000,0.000009,-0.174167,0.000000,0.000000,0.083333,0.000000
5,0.000000,0.000000,0.000000,131.333344,0.000014,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.023667,0.161258,123.500000,0.000019,0.023667,0.000000,0.000000,0.000000,0.000000
7,0.000000,-0.020667,0.122526,255.500000,0.000013,-0.020667,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,579.333374,0.000010,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,304.166687,0.000014,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,542.000000,0.000011,0.000000,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Энэ алгоритмыг шийдэж буй асуудлыг тайлбарлана уу. 
Answer:
None 
Response:
Алгоритм шийдэж буй асуудлын тайлбар

Алгоритм шийдэж буй асуудлууд нь байршингийн холоос шийдэхийн оролцоог хүлээлцдэг нэгтэй хоорондоо ширхэцэн үйлдэлүүдийн тохиролцоо бүтэц бөгөөд энэ боловсролоор компьютерийн жишгээр стратегийн шийдэх асуудлыг шийдэх жишгээр хэрэгжүүлнэ.

Мөн тийм асуудлуудыг алгоритмын тайлбар гүйцэтгээдгийн илчлээр давхарда zero-sum шийдэх асуудлаар тохирдог.

Алгоритмын шийдэх асуудлуудад саналын гарим шийдэх асуудлын хувьсагшибка хөгжүүлж байршингийн холоос хөтөлцөн үйлдэх оролцоо бид шийдвэр хийх хэдийнэй боловсролоо бодиболсондоо шилдрдэг. 
Extracted:
Алгоритм шийдэж буй асуудлын тайлбар

Алгоритм шийдэж буй асуудлууд нь байршингийн холоос шийдэхийн оролцоог хүлээлцдэг нэгтэй хоорондоо ширхэцэн үйлдэлүүдийн тохиролцоо бүтэц бөгөөд энэ боловсролоор компьютерийн жишгээр стратегийн шийдэх асуудлыг шийдэх жишгээр хэрэгжүүлнэ.

Мөн тийм асуудлуудыг алгоритмын

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:51<00:00, 51.81s/it, est. speed input: 0.75 toks/s, output: 17.01 toks/s]


'**Calculating Pi using Python**\n\nPi (π) is a mathematical constant representing the ratio of a circle\'s circumference to its diameter. Here\'s a simple and efficient way to calculate an approximation of pi using Python.\n\n### Using the Monte Carlo Method\n\nThe Monte Carlo method is a computational algorithm that uses random sampling to approximate a value. In this case, we can use it to estimate pi by generating random points within a square and checking if they fall inside a quarter-circle inscribed within it.\n\n```python\nimport random\nimport math\n\ndef estimate_pi(num_samples):\n    """\n    Estimate the value of pi using the Monte Carlo method.\n\n    Args:\n    num_samples (int): The number of random points to generate.\n\n    Returns:\n    float: An approximation of pi.\n    """\n    points_inside_circle = 0\n\n    for _ in range(num_samples):\n        x, y = random.random(), random.random()\n        distance = x**2 + y**2\n        if distance <= 1:\n            points_i

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.17s/it, est. speed input: 2.63 toks/s, output: 15.80 toks/s]


'Calculating pi to a high degree of accuracy is a complex task that requires a large amount of computational power. However, I can provide you with an approximate value of pi or show you a simple method to calculate it.\n\nOne of the simplest methods to calculate pi is the Leibniz formula, which is an infinite series:\n\npi/4 = 1 - 1/3 + 1/5 - 1/7 + 1/9 - ...\n\nThis series can be used to calculate an approximation of pi.\n\n<reasoning>\npi = 4 * (1 - 1/3 + 1/5 - 1/7 + 1/9 - ...)\n</reasoning>\n\nThis is a simple, yet effective method to calculate pi. However, the more terms you use, the more accurate the result will be.\n\nTo calculate pi to a high degree of accuracy, you would need to use a computer program to perform the calculation.\n\n<answer>\n3.141592653589793 (approximately)\n</answer>\n\nFor a more accurate result, I can provide you with a Python code snippet to calculate pi:\n\n```python\nimport math\n\ndef calculate_pi(n):\n    pi = 0.0\n    for i in range(n):\n        pi +=

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )